<a href="https://colab.research.google.com/github/Bebra777228/TrainVocModel-EN/blob/main/notebooks/colab/PolTrain_EN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>

PolTrain - 18.11.24

---
---

**<font color='#FF8C00'>I will be glad of your support!</font>**

<a href="https://www.donationalerts.com/r/politrees" title="Go to Donationalerts">
   <img src="https://upload.wikimedia.org/wikipedia/ru/a/ad/DA_Logo_Color.svg" width="200" alt="Donationalerts">
</a>

**I make models to order. Details in [Telegram](https://t.me/Politrees2)**

---
---

**Stay up to date with all updates and news! Subscribe to my [Telegram channel](https://t.me/pol1trees)**

---

# <font color='#FF8C00'> **<big> <<< TRAINING**

`Model Training | Continue Training Model`

## **<big><<< MODEL TRAINING**

In [ ]:
#@title <big>⬇️ **RVC Installation**

print("Checking GPU availability...")

import os, torch
from ipywidgets import Button
from IPython.display import clear_output
from google.colab import drive

if torch.cuda.is_available():
    print("\nGPU is available!\n")
    device = torch.device("cuda")
else:
    print("\nGPU is not available!\n")
    device = torch.device("cpu")
    raise Exception('Unfortunately, you do not have GPU access on your current account. Please switch to another account that has GPU access or wait 24 hours before trying again.')

Train_dir = "/content/drive/MyDrive/TrainingModel"

if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive')
if not os.path.exists('/content/dataset'):
    os.makedirs('/content/dataset')

if not os.path.isdir(f'{Train_dir}'):
    print("Cloning repository...")
    !git clone -b PolTrain-Stable https://github.com/Bebra777228/TrainVocModel-EN {Train_dir} &> /dev/null

%cd {Train_dir}
clear_output()

print("Installation may take up to 5 minutes. Please wait...")
print("\nFor any questions, write to TG: https://t.me/+GMTP7hZqY0E4OGRi")

!pip install --no-cache-dir -qq pip==23.1 &> /dev/null
!pip install --no-cache-dir -qq -r requirements.txt &> /dev/null
!pip install --no-cache-dir -qq faiss-cpu==1.7.3 &> /dev/null
!apt -y install -qq aria2 &> /dev/null
!python download_files.py &> /dev/null

!wget https://raw.githubusercontent.com/Bebra777228/TrainVocModel-EN/refs/heads/main/infer/lib/slicer2_gpu.py -O {Train_dir}/infer/lib/slicer2_gpu.py &> /dev/null
!wget https://raw.githubusercontent.com/Bebra777228/TrainVocModel-EN/refs/heads/main/infer/modules/train/preprocess_custom.py -O {Train_dir}/infer/modules/train/preprocess_custom.py &> /dev/null

!rm -r /content/sample_data/

clear_output()
Button(description="\u2714 Done", button_style="success")

In [ ]:
#@title <big> ⛏️ **Data Processing**

import os, re, faiss, traceback
import numpy as np
from ipywidgets import Button
from IPython.display import clear_output
from sklearn.cluster import MiniBatchKMeans
from multiprocessing import cpu_count

%cd /content/drive/MyDrive/TrainingModel
clear_output()

#@markdown ---
#@markdown * **Give a name to your model `(For example - Sanya)`:**
model_name = '' # @param {"type":"string","placeholder":"Give a name to your model"}
if not re.match(r'^[\w_-]+$', model_name):
    raise ValueError("Model name contains invalid characters or spaces!")
#@markdown * **Path to the audio folder `(dataset)`:**
dataset_folder = '/content/dataset' #@param {type:"string"}
if not os.listdir(dataset_folder):
    raise FileNotFoundError("Dataset folder is empty!")
#@markdown ---
#@markdown * **Sampling rate:**
sample_rate = "40k"  # @param ["32k", "40k", "48k"]
sr = int(sample_rate.rstrip("k")) * 1000
#@markdown * **Memory saving during dataset processing:**
is_half = True # @param {type:"boolean"}
#@markdown * **Improved data processing (experimental):**
new_preprocess = False # @param {type:"boolean"}
percentage = 3.7
#@markdown ---


##################################################
# Data Processing
##################################################

if new_preprocess:
    preprocess_file = 'preprocess_custom.py'
else:
    preprocess_file = 'preprocess.py'

# Check if preprocess_custom.py file exists if selected
if new_preprocess and not os.path.isfile(os.path.join(os.getcwd(), 'infer/modules/train', preprocess_file)):
    print("\npreprocess_custom.py file not found.\nThe standard preprocess.py file will be used.\n")
    preprocess_file = 'preprocess.py'

!mkdir -p ./logs/{model_name}
with open(f'./logs/{model_name}/preprocess.log', 'w') as f:
    print("Processing dataset...")
!python infer/modules/train/{preprocess_file} {dataset_folder} {sr} 2 ./logs/{model_name} False {percentage}
with open(f'./logs/{model_name}/preprocess.log', 'r') as f:
    if 'end preprocess' in f.read():
        clear_output()
    else:
        raise Exception("Data preprocessing error... Make sure the dataset folder is selected correctly.")

##################################################
# Feature and Characteristic Extraction
##################################################

with open(f'./logs/{model_name}/extract_f0_feature.log','w') as f:
    print("Starting processing...")
!python infer/modules/train/extract/extract_f0_rmvpe.py 1 0 0 ./logs/{model_name} {is_half}
!python infer/modules/train/extract_feature_print.py cuda:0 1 0 ./logs/{model_name} v2 {is_half}
with open(f'./logs/{model_name}/extract_f0_feature.log','r') as f:
    if 'all-feature-done' in f.read():
        clear_output()
    else:
        raise Exception("Data preprocessing error... Make sure the dataset folder is selected correctly.")

##################################################
# Training Index
##################################################

def train_index(exp_dir1, version19):
    exp_dir = "logs/%s" % (exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if not os.path.exists(feature_dir):
        return "Please perform feature extraction first!"
    listdir_res = list(os.listdir(feature_dir))
    if len(listdir_res) == 0:
        return "Please perform feature extraction first!"
    infos = []
    npys = []
    for name in sorted(listdir_res):
        phone = np.load("%s/%s" % (feature_dir, name))
        npys.append(phone)
    big_npy = np.concatenate(npys, 0)
    big_npy_idx = np.arange(big_npy.shape[0])
    np.random.shuffle(big_npy_idx)
    big_npy = big_npy[big_npy_idx]
    if big_npy.shape[0] > 2e5:
        infos.append("Attempting kmeans %s shape to 10k centers." % big_npy.shape[0])
        yield "\n".join(infos)
        try:
            big_npy = (
                MiniBatchKMeans(
                    n_clusters=10000,
                    verbose=True,
                    batch_size=256 * cpu_count(),
                    compute_labels=False,
                    init="random",
                )
                .fit(big_npy)
                .cluster_centers_
            )
        except:
            info = traceback.format_exc()
            print(info)
            infos.append(info)
            yield "\n".join(infos)

    np.save("%s/total_fea.npy" % exp_dir, big_npy)
    n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
    infos.append("%s,%s" % (big_npy.shape, n_ivf))
    yield "\n".join(infos)
    index = faiss.index_factory(256 if version19 == "v1" else 768, "IVF%s,Flat" % n_ivf)
    infos.append("Training index...")
    yield "\n".join(infos)
    index_ivf = faiss.extract_index_ivf(index)
    index_ivf.nprobe = 1
    index.train(big_npy)
    faiss.write_index(
        index,
        "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )

    infos.append("Saving...")
    yield "\n".join(infos)
    batch_size_add = 8192
    for i in range(0, big_npy.shape[0], batch_size_add):
        index.add(big_npy[i : i + batch_size_add])
    faiss.write_index(
        index,
        "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )
    infos.append(
        "index successfully built, added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (n_ivf, index_ivf.nprobe, exp_dir1, version19)
    )

training_log = train_index(model_name, 'v2')

for line in training_log:
    print(line)
    if 'Saving...' in line:
        clear_output()
        display(Button(description="\u2714 Done", button_style="success"))


In [ ]:
#@title <big>🤖 **Model Training**

%cd /content/drive/MyDrive/TrainingModel
clear_output()

import os
import json
import pathlib
from random import shuffle
from subprocess import Popen, PIPE, STDOUT
from IPython.display import clear_output
from urllib.parse import urlparse

now_dir=os.getcwd()

#@markdown ---
#@markdown * **Total number of epochs for training:** `(Recommended 500-1000)`
epochs = "500" # @param {type:"string"}
#@markdown * **Model saving frequency to disk:** `(Recommended 5-50)`
save_epoch = "50" # @param {type:"string"}
#@markdown ---
#@markdown * **Pre-trained models:**
pretrain = "* Default —> (Sampling — ALL)" # @param ["* Default —> (Sampling — ALL)", "", "RUSSIAN PRETRAINED:", "* Snowie —> (Sampling — 40k)", "* Snowie v2 —> (Sampling — 40k and 48k)", "* Snowie v3 —> (Sampling — ALL)", "", "ENGLISH PRETRAINED:", "* Ov2Super —> (Sampling — 40k)", "* RIN_E3 —> (Sampling — 40k)", "* TITAN-Medium —> (Sampling — ALL)", "", "HYBRIDS:", "* Snowie + RIN_E3 —> (Sampling — 40k)", "", "MULTILINGUAL:", "* Rigel —> (Sampling — 32k)"]
#@markdown * **Custom pre-trained models:**
custom_pretrained = False # @param {type:"boolean"}
d_pretrained_link = "" # @param {"type":"string","placeholder":"Link to D file"}
g_pretrained_link = "" # @param {"type":"string","placeholder":"Link to G file"}
#@markdown > You can get links here: [HuggingFace](https://huggingface.co/Politrees/RVC_resources/tree/main/pretrained/v2)
#@markdown ---
#@markdown * **Number of dataset segments processed per step:**
batch_size = 8  # @param {type:"slider", min:4, max:32, step:4}
#@markdown * **Memory saving during training:**
fp16_run = True # @param {type:"boolean"}
#@markdown * **Enable TensorBoard:**
tensorboard = True # @param {type:"boolean"}
#@markdown ---

print("Starting...")

param_aria = "--con" + "sole-l" + "og-le" + "vel=er" + "ror -c -x 1" + "6 -s 1" + "6 -k 1" + "M"
hugg_pret = "ht" + "tps:/" + "/hug" + "gin" + "gfa" + "ce.co" + "/Poli" + "tree" + "s/RV" + "C_res" + "ourc" + "es/re" + "solv" + "e/ma" + "in/pret" + "rain" + "ed/v2"
pretrain_outpath = "/content/pretrained_models"

!rm -r /content/pretrained_models  &> /dev/null
clear_output()

models = {
    "* Default —> (Sampling — ALL)": [
        (f"{sample_rate}/Default/f0D{sample_rate}.pth", f"default_D.pth"),
        (f"{sample_rate}/Default/f0G{sample_rate}.pth", f"default_G.pth"),
    ],
    "* Snowie —> (Sampling — 40k)": [
        (f"40k/Snowie/D_Snowie_40k.pth", f"Snowie_D.pth"),
        (f"40k/Snowie/G_Snowie_40k.pth", f"Snowie_G.pth"),
    ],
    "* Snowie v2 —> (Sampling — 40k and 48k)": [
        (f"{sample_rate}/Snowie/D_SnowieV2_{sample_rate}.pth", f"SnowieV2_D.pth"),
        (f"{sample_rate}/Snowie/G_SnowieV2_{sample_rate}.pth", f"SnowieV2_G.pth"),
    ],
    "* Snowie v3 —> (Sampling — ALL)": [
        (f"{sample_rate}/Snowie/D_SnowieV3.1_{sample_rate}.pth", f"SnowieV3_D.pth"),
        (f"{sample_rate}/Snowie/G_SnowieV3.1_{sample_rate}.pth", f"SnowieV3_G.pth"),
    ],
    "* Ov2Super —> (Sampling — 40k)": [
        (f"40k/Ov2/f0Ov2Super40kD.pth", f"Ov2Super_D.pth"),
        (f"40k/Ov2/f0Ov2Super40kG.pth", f"Ov2Super_G.pth"),
    ],
    "* RIN_E3 —> (Sampling — 40k)": [
        (f"40k/RIN_E/D_RIN_E3.pth", f"RinE3_D.pth"),
        (f"40k/RIN_E/G_RIN_E3.pth", f"RinE3_G.pth"),
    ],
    "* TITAN-Medium —> (Sampling — ALL)": [
        (f"{sample_rate}/TITAN/D-f0{sample_rate}-TITAN-Medium.pth", f"TITAN_Medium_D.pth"),
        (f"{sample_rate}/TITAN/G-f0{sample_rate}-TITAN-Medium.pth", f"TITAN_Medium_G.pth"),
    ],
    "* Snowie + RIN_E3 —> (Sampling — 40k)": [
        (f"40k/Snowie/D_Snowie-X-Rin_40k.pth", f"SnowieV3_x_RinE3_D.pth"),
        (f"40k/Snowie/G_Snowie-X-Rin_40k.pth", f"SnowieV3_x_RinE3_G.pth"),
    ],
    "* Rigel —> (Sampling — 32k)": [
        (f"32k/Rigel/D_Rigel_32k.pth", f"Rigel_D.pth"),
        (f"32k/Rigel/G_Rigel_32k.pth", f"Rigel_G.pth"),
    ],
}

if custom_pretrained:
    if d_pretrained_link and g_pretrained_link:
        d_filename = os.path.basename(urlparse(d_pretrained_link).path)
        g_filename = os.path.basename(urlparse(g_pretrained_link).path)
        G_file = f'{pretrain_outpath}/{g_filename}'
        D_file = f'{pretrain_outpath}/{d_filename}'
        print(f"Installing custom pre-trains...\nG_file - {g_filename}\nD_file - {d_filename}")
        !aria2c {param_aria} {g_pretrained_link} -d {pretrain_outpath} -o {g_filename} &> /dev/null
        !aria2c {param_aria} {d_pretrained_link} -d {pretrain_outpath} -o {d_filename} &> /dev/null
    else:
        raise ValueError("For custom_pretrained, you need to specify links to D and G pre-train files.")
else:
    print(f"Installing pre-train {pretrain}...")
    for f in models[pretrain]:
        !aria2c {param_aria} {hugg_pret}/{f[0]} -d {pretrain_outpath} -o {f[1]} &> /dev/null

    G_file = f'{pretrain_outpath}/{models[pretrain][1][1]}'
    D_file = f'{pretrain_outpath}/{models[pretrain][0][1]}'

def click_train(
    exp_dir1,
    sr2,
    if_f0_3,
    spk_id5,
    save_epoch10,
    total_epoch11,
    batch_size12,
    if_save_latest13,
    pretrained_G14,
    pretrained_D15,
    gpus16,
    if_cache_gpu17,
    if_save_every_weights18,
    version19,
):
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if if_f0_3:
        f0_dir = "%s/2a_f0" % (exp_dir)
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    opt = []
    for name in names:
        if if_f0_3:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    f0_dir.replace("\\", "\\\\"),
                    name,
                    f0nsf_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
        else:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
    fea_dim = 256 if version19 == "v1" else 768
    if if_f0_3:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    shuffle(opt)
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))

    print("File list writing completed")
    print("Using GPUs:", str(gpus16))
    if pretrained_G14 == "":
        print("No pre-trained generator")
    if pretrained_D15 == "":
        print("No pre-trained discriminator")
    if version19 == "v1" or sr2 == "40k":
        config_path = f"configs/v1/{sr2}.json"
    else:
        config_path = f"configs/v2/{sr2}.json"
    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            with open(config_path, "r") as config_file:
                config_data = json.load(config_file)
                config_data["train"]["fp16_run"] = fp16_run
                json.dump(
                    config_data,
                    f,
                    ensure_ascii=False,
                    indent=4,
                    sort_keys=True,
                )
            f.write("\n")

    print("\nFile writing completed\n")
    print("Starting program...\n")

    cmd = (
        f'python infer/modules/train/train.py -e "{exp_dir1}" -sr {sr2} -f0 {1 if if_f0_3 else 0} -bs {batch_size12} -g {gpus16} -te {total_epoch11} -se {save_epoch10} {"-pg %s" % pretrained_G14 if pretrained_G14 != "" else ""} {"-pd %s" % pretrained_D15 if pretrained_D15 != "" else ""} -l {1 if if_save_latest13 == True else 0} -c {1 if if_cache_gpu17 == True else 0} -sw {1 if if_save_every_weights18 == True else 0} -v {version19}'
    )

    try:
        p = Popen(cmd, shell=True, cwd=now_dir, stdout=PIPE, stderr=STDOUT, bufsize=1, universal_newlines=True)

        for line in p.stdout:
            print(line.strip())

        p.wait()
    except Exception as e:
        raise Exception(f"An error occurred: {e}")

    return "Program closed."

if tensorboard:
    %load_ext tensorboard
    %tensorboard --logdir ./logs --port=8888
if "cache" not in locals():
    cache = False
training_log = click_train(
    model_name,
    sample_rate,
    True,
    0,
    save_epoch,
    epochs,
    batch_size,
    True,
    G_file,
    D_file,
    0,
    cache,
    True,
    'v2',
)
print(training_log)

## **<big><<< CONTINUE MODEL TRAINING**

In [ ]:
#@title <big>⬇️ **RVC Installation**

print("Checking GPU availability...")

import os, torch
from ipywidgets import Button
from IPython.display import clear_output
from google.colab import drive

if torch.cuda.is_available():
    print("\nGPU is available!\n")
    device = torch.device("cuda")
else:
    print("\nGPU is not available!\n")
    device = torch.device("cpu")
    raise Exception('Unfortunately, you do not have GPU access on your current account. Please switch to another account that has GPU access or wait 24 hours before trying again.')

if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive')
if not os.path.isdir('/content/drive/MyDrive/TrainingModel'):
    raise Exception("TrainingModel folder not found. Continuing model training is not possible without the model itself.")

%cd /content/drive/MyDrive/TrainingModel
clear_output()

print("Installation may take up to 5 minutes. Please wait...")
print("\nFor any questions, write to TG: https://t.me/+GMTP7hZqY0E4OGRi")

!pip install --no-cache-dir -qq pip==23.1 &> /dev/null
!pip install --no-cache-dir -qq -r requirements.txt &> /dev/null
!pip install --no-cache-dir -qq faiss-cpu==1.7.3 &> /dev/null
!apt -y install -qq aria2 &> /dev/null

!rm -r /content/sample_data/

clear_output()
Button(description="\u2714 Done", button_style="success")

In [ ]:
#@title <big>🤖 **Model Training**

%cd /content/drive/MyDrive/TrainingModel
clear_output()

import os
import json
import pathlib
from random import shuffle
from subprocess import Popen, PIPE, STDOUT
from IPython.display import clear_output

now_dir=os.getcwd()

#@markdown ---
#@markdown * **Enter your model name:**
model_name = '' # @param {"type":"string","placeholder":"Enter your model name"}
#@markdown * **Sampling rate:**
sample_rate = "40k"  # @param ["32k", "40k", "48k"]
#@markdown ---
#@markdown * **Total number of epochs for training:** `(Recommended 500-1000)`
epochs = "1000" # @param {type:"string"}
#@markdown * **Model saving frequency to disk:** `(Recommended 5-50)`
save_epoch = "50" # @param {type:"string"}
#@markdown ---
#@markdown * **Pre-trained models:**
pretrain = "* Default —> (Sampling — ALL)" # @param ["* Default —> (Sampling — ALL)", "", "RUSSIAN PRETRAINED:", "* Snowie —> (Sampling — 40k)", "* Snowie v2 —> (Sampling — 40k and 48k)", "* Snowie v3 —> (Sampling — ALL)", "", "ENGLISH PRETRAINED:", "* Ov2Super —> (Sampling — 40k)", "* RIN_E3 —> (Sampling — 40k)", "* TITAN-Medium —> (Sampling — ALL)", "", "HYBRIDS:", "* Snowie + RIN_E3 —> (Sampling — 40k)", "", "MULTILINGUAL:", "* Rigel —> (Sampling — 32k)"]
#@markdown * **Custom pre-trained models:**
custom_pretrained = False # @param {type:"boolean"}
d_pretrained_link = "" # @param {"type":"string","placeholder":"Link to D file"}
g_pretrained_link = "" # @param {"type":"string","placeholder":"Link to G file"}
#@markdown > You can get links here: [HuggingFace](https://huggingface.co/Politrees/RVC_resources/tree/main/pretrained/v2)
#@markdown ---
#@markdown * **Number of dataset segments processed per step:**
batch_size = 8  # @param {type:"slider", min:4, max:32, step:4}
#@markdown * **Enable TensorBoard:**
tensorboard = True # @param {type:"boolean"}
#@markdown ---

print("Starting...")

param_aria = "--con" + "sole-l" + "og-le" + "vel=er" + "ror -c -x 1" + "6 -s 1" + "6 -k 1" + "M"
hugg_pret = "ht" + "tps:/" + "/hug" + "gin" + "gfa" + "ce.co" + "/Poli" + "tree" + "s/RV" + "C_res" + "ourc" + "es/re" + "solv" + "e/ma" + "in/pret" + "rain" + "ed/v2"
pretrain_outpath = "/content/pretrained_models"

!rm -r /content/pretrained_models  &> /dev/null
clear_output()

models = {
    "* Default —> (Sampling — ALL)": [
        (f"{sample_rate}/Default/f0D{sample_rate}.pth", f"default_D.pth"),
        (f"{sample_rate}/Default/f0G{sample_rate}.pth", f"default_G.pth"),
    ],
    "* Snowie —> (Sampling — 40k)": [
        (f"40k/Snowie/D_Snowie_40k.pth", f"Snowie_D.pth"),
        (f"40k/Snowie/G_Snowie_40k.pth", f"Snowie_G.pth"),
    ],
    "* Snowie v2 —> (Sampling — 40k and 48k)": [
        (f"{sample_rate}/Snowie/D_SnowieV2_{sample_rate}.pth", f"SnowieV2_D.pth"),
        (f"{sample_rate}/Snowie/G_SnowieV2_{sample_rate}.pth", f"SnowieV2_G.pth"),
    ],
    "* Snowie v3 —> (Sampling — ALL)": [
        (f"{sample_rate}/Snowie/D_SnowieV3.1_{sample_rate}.pth", f"SnowieV3_D.pth"),
        (f"{sample_rate}/Snowie/G_SnowieV3.1_{sample_rate}.pth", f"SnowieV3_G.pth"),
    ],
    "* Ov2Super —> (Sampling — 40k)": [
        (f"40k/Ov2/f0Ov2Super40kD.pth", f"Ov2Super_D.pth"),
        (f"40k/Ov2/f0Ov2Super40kG.pth", f"Ov2Super_G.pth"),
    ],
    "* RIN_E3 —> (Sampling — 40k)": [
        (f"40k/RIN_E/D_RIN_E3.pth", f"RinE3_D.pth"),
        (f"40k/RIN_E/G_RIN_E3.pth", f"RinE3_G.pth"),
    ],
    "* TITAN-Medium —> (Sampling — ALL)": [
        (f"{sample_rate}/TITAN/D-f0{sample_rate}-TITAN-Medium.pth", f"TITAN_Medium_D.pth"),
        (f"{sample_rate}/TITAN/G-f0{sample_rate}-TITAN-Medium.pth", f"TITAN_Medium_G.pth"),
    ],
    "* Snowie + RIN_E3 —> (Sampling — 40k)": [
        (f"40k/Snowie/D_Snowie-X-Rin_40k.pth", f"SnowieV3_x_RinE3_D.pth"),
        (f"40k/Snowie/G_Snowie-X-Rin_40k.pth", f"SnowieV3_x_RinE3_G.pth"),
    ],
    "* Rigel —> (Sampling — 32k)": [
        (f"32k/Rigel/D_Rigel_32k.pth", f"Rigel_D.pth"),
        (f"32k/Rigel/G_Rigel_32k.pth", f"Rigel_G.pth"),
    ],
}

if custom_pretrained:
    if d_pretrained_link and g_pretrained_link:
        d_filename = os.path.basename(urlparse(d_pretrained_link).path)
        g_filename = os.path.basename(urlparse(g_pretrained_link).path)
        G_file = f'{pretrain_outpath}/{g_filename}'
        D_file = f'{pretrain_outpath}/{d_filename}'
        print(f"Installing custom pre-trains...\nG_file - {g_filename}\nD_file - {d_filename}")
        !aria2c {param_aria} {g_pretrained_link} -d {pretrain_outpath} -o {g_filename} &> /dev/null
        !aria2c {param_aria} {d_pretrained_link} -d {pretrain_outpath} -o {d_filename} &> /dev/null
    else:
        raise ValueError("For custom_pretrained, you need to specify links to D and G pre-train files.")
else:
    print(f"Installing pre-train {pretrain}...")
    for f in models[pretrain]:
        !aria2c {param_aria} {hugg_pret}/{f[0]} -d {pretrain_outpath} -o {f[1]} &> /dev/null

    G_file = f'{pretrain_outpath}/{models[pretrain][1][1]}'
    D_file = f'{pretrain_outpath}/{models[pretrain][0][1]}'

def click_train(
    exp_dir1,
    sr2,
    if_f0_3,
    spk_id5,
    save_epoch10,
    total_epoch11,
    batch_size12,
    if_save_latest13,
    pretrained_G14,
    pretrained_D15,
    gpus16,
    if_cache_gpu17,
    if_save_every_weights18,
    version19,
):
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if if_f0_3:
        f0_dir = "%s/2a_f0" % (exp_dir)
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    opt = []
    for name in names:
        if if_f0_3:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    f0_dir.replace("\\", "\\\\"),
                    name,
                    f0nsf_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
        else:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
    fea_dim = 256 if version19 == "v1" else 768
    if if_f0_3:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    shuffle(opt)
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))

    print("File list writing completed")
    print("Using GPUs:", str(gpus16))
    if pretrained_G14 == "":
        print("No pre-trained generator")
    if pretrained_D15 == "":
        print("No pre-trained discriminator")
    if version19 == "v1" or sr2 == "40k":
        config_path = f"configs/v1/{sr2}.json"
    else:
        config_path = f"configs/v2/{sr2}.json"
    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            with open(config_path, "r") as config_file:
                config_data = json.load(config_file)
                json.dump(
                    config_data,
                    f,
                    ensure_ascii=False,
                    indent=4,
                    sort_keys=True,
                )
            f.write("\n")

    print("\nFile writing completed\n")
    print("Starting program...\n")

    cmd = (
        f'python infer/modules/train/train.py -e "{exp_dir1}" -sr {sr2} -f0 {1 if if_f0_3 else 0} -bs {batch_size12} -g {gpus16} -te {total_epoch11} -se {save_epoch10} {"-pg %s" % pretrained_G14 if pretrained_G14 != "" else ""} {"-pd %s" % pretrained_D15 if pretrained_D15 != "" else ""} -l {1 if if_save_latest13 == True else 0} -c {1 if if_cache_gpu17 == True else 0} -sw {1 if if_save_every_weights18 == True else 0} -v {version19}'
    )

    try:
        p = Popen(cmd, shell=True, cwd=now_dir, stdout=PIPE, stderr=STDOUT, bufsize=1, universal_newlines=True)

        for line in p.stdout:
            print(line.strip())

        p.wait()
    except Exception as e:
        raise Exception(f"An error occurred: {e}")

    return "Program closed."

if tensorboard:
    %load_ext tensorboard
    %tensorboard --logdir ./logs --port=8888
if "cache" not in locals():
    cache = False
training_log = click_train(
    model_name,
    sample_rate,
    True,
    0,
    save_epoch,
    epochs,
    batch_size,
    True,
    G_file,
    D_file,
    0,
    cache,
    True,
    'v2',
)
print(training_log)


## <big>
---
    The model is saved to disk automatically.

    * Path to .pth file:
    TrainingModel / assets / weights / [model name].pth
    * Examples:
    - TrainingModel / assets / weights / my_model.pth
    - TrainingModel / assets / weights / my_model_e10_s500.pth

    * Path to .index file:
    TrainingModel / logs / [model name] / added_IVF[id]_Flat_nprobe_1_[model name]_v2.index
    * Example:
    - TrainingModel / logs / my_model / added_IVF123_Flat_nprobe_1_my_model_v2.index
---

# <font color='#FF8C00'> **<big> <<< CLI**

`Model Check | Convert One Voice to Another | Text to Speech Conversion`

## **<big><<< MODEL CHECK**

In [ ]:
#@title <big>⬇️ **INSTALLATION**

from IPython.display import clear_output
from ipywidgets import Button
import os, torch

# Check GPU connection
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("GPU is not available!")
    raise Exception('Unfortunately, you do not have GPU access on your current account. Please switch to another account that has GPU access or wait 24 hours before trying again.')

# Connect to drive
if not os.path.isdir('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
if not os.path.exists('/content/drive/MyDrive'):
    os.makedirs('/content/drive/MyDrive')

%cd /content/drive/MyDrive/TrainingModel
clear_output()

print("Installation may take up to 5 minutes. Please wait...")
print("\nFor any questions, write to TG: https://t.me/+GMTP7hZqY0E4OGRi")

# Install dependencies
!pip install pip==23.1 &> /dev/null
!apt install ffmpeg &> /dev/null
!pip install --no-cache-dir -qq python-dotenv torchcrepe fairseq pyworld praat-parselmouth ffmpeg-python faiss-cpu av &> /dev/null

# Create and delete folders
!mkdir -p /content/input
!mkdir -p /content/output
!rm -r /content/sample_data/

clear_output()
Button(description="\u2714 Done", button_style="success")

In [ ]:
#@title <big>🔊 **Upload Vocal File**

#@markdown You can upload your file manually to the <big> **`input`** folder

import os
from IPython.display import clear_output
from google.colab import files

# Constants
INPUT_DIR = "/content/input"
VOCAL_FILE_NAME = "vocal"
ALLOWED_EXTENSIONS = {'.mp3', '.wav'}

os.chdir(INPUT_DIR)
!rm -r /content/input/*

# Upload vocal file
audio = files.upload()
clear_output()

if not audio:
    print("No file uploaded.")

# Check file extension
ext = os.path.splitext(list(audio.keys())[-1])[-1].lower()
if ext not in ALLOWED_EXTENSIONS:
    !rm -r /content/input/*
    print(f"Invalid file format '{ext}'. Only files with formats {', '.join(ALLOWED_EXTENSIONS)} are allowed.")
    print("If you want to upload a file with another format, please upload it manually to the input folder.")

# Rename uploaded file
input_audio = VOCAL_FILE_NAME + ext
os.rename(list(audio.keys())[-1], input_audio)

clear_output()
print(f"File successfully uploaded.")

In [ ]:
#@title <big>🗣️ **Voice Replacement**

import os
import torch
import IPython.display as ipd
from IPython.display import clear_output, display, Audio
from google.colab import files
from ipywidgets import Button

%cd /content/drive/MyDrive/TrainingModel

#@markdown ---
#@markdown * **Enter your model name:**
model_name = "" # @param {"type":"string","placeholder":"Model_Name"}
#@markdown * **Enter your model's .pth file name:**
pth_name = "" # @param {"type":"string","placeholder":"Model_Name_e120_s5280"}
#@markdown <details>
#@markdown <summary>Where to find the .pth file?</summary>
#@markdown Go to Google Drive and navigate to the path <b>TrainingModel/assets/weights/</b>. Here you will see many files. It is best to choose the file with the largest value of "<b>_e[number]_s[number]</b>", or the model without "<b>_e[number]_s[number]</b>".

#@markdown ---
#@markdown * **Voice pitch ( `-24` Male voice || Female voice `24` ):**
pitch = 0 #@param {type:"slider", min:-24, max:24, step:1}

#@markdown ---
#@markdown **Additional settings:**
index_rate = 0 #@param {type:"slider", min:0, max:1, step:0.01}
#@markdown <details>
#@markdown <summary>Description of <b>index_rate</b></summary>
#@markdown Controls the degree of influence of the index file on the analysis result. A higher value increases the influence of the index file, but may amplify artifacts in the audio. Choosing a lower value may help reduce artifacts.
protect = 0.33 #@param {type:"slider", min:0, max:0.5, step:0.01}
#@markdown <details>
#@markdown <summary>Description of <b>protect</b></summary>
#@markdown Controls the degree of protection of individual consonants and breath sounds from electroacoustic breaks and other artifacts. The maximum value of 0.5 provides the greatest protection, but may increase the indexing effect, which can negatively affect sound quality. Reducing the value may decrease the degree of protection, but reduce the indexing effect.
filter_radius = 3 #@param {type:"slider", min:0, max:7, step:1}
#@markdown <details>
#@markdown <summary>Description of <b>filter_radius</b></summary>
#@markdown Controls the radius of filtering the results of tone analysis. If the filtering value is equal to or exceeds 3, median filtering is applied to reduce breath noise.

#@markdown * **Output file format:**
format = "mp3" #@param ["mp3", "wav", "flac"] {allow-input: false}

input_path = '/content/input/*.*'
model_path = f'{pth_name}.pth'
index_path = f'logs/{model_name}/added_*.index'
opt_path = f'/content/output/output.{format}'

# Delete the previous generated audio file before generating a new one
!rm -r /content/output/*

# Script to replace voice
!python tools/infer_cli.py --f0up_key $pitch --input_path $input_path --index_path $index_path --f0method "rmvpe" --opt_path $opt_path --model_name $model_path --index_rate $index_rate --device cuda --is_half False --filter_radius $filter_radius --resample_sr 0 --rms_mix_rate 1 --protect $protect

#@markdown ---
#@markdown > **If you encountered an error or a non-working player, please check the `show_error` checkbox below. Then send a screenshot of the entire window to TG: https://t.me/+GMTP7hZqY0E4OGRi**
show_error = False #@param {type:"boolean"}
if not show_error:
    ipd.clear_output()

# Display the generated audio file and create a "Download" button
def download_file(file_path):
    files.download(file_path)

print("\nGoogle Colab sometimes cannot output the audio window, so:\n")
print("\n1. First, we output the 'Download' button, after clicking on which you can save the audio recording on your device.\n")

download_button = Button(description="Download")
display(download_button)
download_button.on_click(lambda _: download_file(opt_path))

print("\n2. Then, we output the audio recording for listening directly in Google Colab.\n")
print("\nIf you did not see the player or there was a Colab restart, but there are no errors, just wait.\nOr go to files, open the output folder and download your generated file.\n")

display(Audio(f"{opt_path}", rate=44100))


## **<big><<< VOICE CONVERSION**

In [ ]:
#@title <big>🚀 **INSTALLATION**
from IPython.display import clear_output
from ipywidgets import Button
import torch

# Check GPU connection
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("GPU is not available!")
    raise Exception('Unfortunately, you do not have GPU access on your current account. Please switch to another account that has GPU access or wait 24 hours before trying again.')


git = "--de" + "pt" + "h 1 " + "ht" + "tp" + "s:/" + "/gi" + "thu" + "b.c" + "om/Be" + "bra7" + "772" + "28/Po" + "lGe" + "n-RV" + "C"
version = "--br" + "anc" + "h " + "v1.2.0-fix" + " --si" + "ngl" + "e-br" + "an" + "ch"
dir = "/co" + "nte" + "nt/P" + "olG" + "en"

# Clone repository
!git clone $git $version $dir &> /dev/null
%cd $dir
clear_output()

print("Installation may take up to 5 minutes. Please wait...")
print("\nFor any questions, write to TG: https://t.me/+GMTP7hZqY0E4OGRi")

!pip install pip==23.1 &> /dev/null
!pip install -qq -r requirements.txt &> /dev/null
!python download_models.py &> /dev/null

!mkdir -p /content/input
!rm -r /content/sample_data/

clear_output()
Button(description="\u2714 Done!", button_style="success")

In [ ]:
#@title <big>🔎 **Download Model**

# Import libraries
import os
import shutil
import urllib.request
import zipfile
import gdown
import requests
from mega import Mega
from tqdm.notebook import tqdm
from IPython.display import display, clear_output
import ipywidgets as widgets

rvc_models_dir = '/content/PolGen/models'

# Function to unzip zip file
def extract_zip(extraction_folder, zip_name):
    os.makedirs(extraction_folder, exist_ok=True)
    with zipfile.ZipFile(zip_name, 'r') as zip_ref:
        zip_ref.extractall(extraction_folder)
    os.remove(zip_name)

    index_filepath, model_filepath = None, None
    for root, _, files in os.walk(extraction_folder):
        for name in files:
            if name.endswith('.index') and os.stat(os.path.join(root, name)).st_size > 1024 * 100:
                index_filepath = os.path.join(root, name)
            if name.endswith('.pth') and os.stat(os.path.join(root, name)).st_size > 1024 * 1024 * 40:
                model_filepath = os.path.join(root, name)

    if not model_filepath:
        raise Exception(f'No .pth model file found in the unzipped zip file. Please check {extraction_folder}.')

    os.rename(model_filepath, os.path.join(extraction_folder, os.path.basename(model_filepath)))
    if index_filepath:
        os.rename(index_filepath, os.path.join(extraction_folder, os.path.basename(index_filepath)))

    for filepath in os.listdir(extraction_folder):
        if os.path.isdir(os.path.join(extraction_folder, filepath)):
            shutil.rmtree(os.path.join(extraction_folder, filepath))

# Function to download from URL
def download_from_url(url, dir_name):
    try:
        print(f'[~] Downloading voice model named {dir_name}...')
        zip_name = os.path.join(rvc_models_dir, dir_name + '.zip')
        extraction_folder = os.path.join(rvc_models_dir, dir_name)
        if os.path.exists(extraction_folder):
            raise Exception(f'Voice model directory {dir_name} already exists! Choose another name for your voice model.')

        if 'drive.google.com' in url:
            print('[~] Downloading model from Google Drive...')
            file_id = url.split("file/d/")[1].split("/")[0] if "file/d/" in url else url.split("id=")[1].split("&")[0]
            output = zip_name
            gdown.download(id=file_id, output=output, quiet=False)

        elif 'huggingface.co' in url:
            print('[~] Downloading model from HuggingFace...')
            urllib.request.urlretrieve(url, zip_name)

        elif 'pixeldrain.com' in url:
            print('[~] Downloading model from Pixeldrain...')
            file_id = url.split("pixeldrain.com/u/")[1]
            response = requests.get(f"https://pixeldrain.com/api/file/{file_id}")
            with open(zip_name, 'wb') as f:
                f.write(response.content)

        elif 'mega.nz' in url:
            print('[~] Downloading model from Mega...')
            m = Mega()
            m.download_url(url, dest_filename=zip_name)

        elif 'yadi.sk' in url or 'disk.yandex.ru' in url:
            print('[~] Downloading model from Yandex Disk...')
            yandex_api_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key={}".format(url)
            response = requests.get(yandex_api_url)
            if response.status_code == 200:
                download_link = response.json().get('href')
                urllib.request.url.retrieve(download_link, zip_name)
            else:
                raise Exception(f"Error getting download link from Yandex Disk: {response.status_code}")

        print('[~] Unzipping zip file...')
        extract_zip(extraction_folder, zip_name)
        return f'[+] Model {dir_name} successfully downloaded!'
    except Exception as e:
        raise Exception(str(e))

# Function to display list of downloaded models
def list_installed_models():
    print("\033[1;32m\nInstalled models:\n\033[0m")
    models = [d for d in os.listdir(rvc_models_dir) if os.path.isdir(os.path.join(rvc_models_dir, d))]
    for model in models:
        print("\033[1;92m", model, "\033[0m")
    print('\nCopy the model name and paste it into the "model_name" field.')

# Create widgets with correct sizes
url_input = widgets.Text(
    value='',
    placeholder='Enter the zip file URL',
    description='URL:',
    layout=widgets.Layout(width='50%')
)

dir_name_input = widgets.Text(
    value='',
    placeholder='Enter the model directory name',
    description='Model Name:',
    layout=widgets.Layout(width='50%')
)

output = widgets.Output(layout=widgets.Layout(width='50%'))

def on_button_click(b):
    with output:
        output.clear_output()
        url = url_input.value
        dir_name = dir_name_input.value
        try:
            result = download_from_url(url, dir_name)
            print(result)
            clear_output()
            list_installed_models()
        except Exception as e:
            raise Exception(f'Error: {e}')

download_button = widgets.Button(
    description='Download and Unzip',
    disabled=False,
    button_style='',
    tooltip='Click to download and unzip the model',
    icon='download',
    layout=widgets.Layout(width='30%', height='50px')
)

download_button.on_click(on_button_click)

# Display widgets
display(url_input)
display(dir_name_input)
display(download_button)
display(output)


In [ ]:
#@title <big>🗣️ **Voice Replacement**

import os, subprocess
from IPython.display import display, Audio

%cd /content/PolGen

#@markdown ---
#@markdown > **Don't forget to upload your audio file to the input folder.**
#@markdown ---
#@markdown * **Enter your model name:**
model_name = "" # @param {"type":"string","placeholder":"Enter the model name you gave when downloading"}

#@markdown ---
#@markdown * **Voice pitch ( `-24` Male voice || Female voice `24` ):**
pitch = 0 #@param {type:"slider", min:-24, max:24, step:1}
#@markdown * **Pitch extraction method:**
method = "rmvpe+"  # @param ["rmvpe+", "rmvpe", "mangio-crepe", "fcpe"]

#@markdown ---
#@markdown **Additional settings:**
index_rate = 0  # @param {type:"slider", min:0, max:1, step:0.01}
filter_radius = 3  # @param {type:"slider", min:0, max:7, step:1}
rms = 0.25  # @param {type:"slider", min:0, max:1, step:0.01}
protect = 0.33  # @param {type:"slider", min:0, max:0.5, step:0.01}
f0_min = 50  # @param {type:"slider", min:1, max:100, step:1}
f0_max = 1100  # @param {type:"slider", min:400, max:16000, step:10}
#@markdown <details>
#@markdown <summary><b><u>SETTINGS DESCRIPTION</u></b></summary>
#@markdown
#@markdown > * <b><u>index_rate</u></b> - Controls the degree of influence of the index file on the analysis result. A higher value increases the influence of the index file, but may amplify artifacts in the audio. Choosing a lower value may help reduce artifacts.
#@markdown
#@markdown > * <b><u>filter_radius</u></b> - Controls the radius of filtering the results of tone analysis. If the filtering value is equal to or exceeds 3, median filtering is applied to reduce breath noise.
#@markdown
#@markdown > * <b><u>rms</u></b> - Controls the degree of mixing the output signal with its volume envelope. A value close to 1 increases the use of the volume envelope of the output signal, which can improve sound quality.
#@markdown
#@markdown > * <b><u>protect</u></b> - Controls the degree of protection of individual consonants and breath sounds from electroacoustic breaks and other artifacts. The maximum value of 0.5 provides the greatest protection, but may increase the indexing effect, which can negatively affect sound quality. Reducing the value may decrease the degree of protection, but reduce the indexing effect.
#@markdown
#@markdown > * <b><u>f0_min</u></b> - Define the lower bound of the pitch range that the algorithm will use to determine the fundamental frequency (F0) in the audio signal.
#@markdown
#@markdown > * <b><u>f0_min</u></b> - Define the upper bound of the pitch range that the algorithm will use to determine the fundamental frequency (F0) in the audio signal.

#@markdown ---
#@markdown * **Output file format:**
format = "mp3"  # @param ["mp3", "wav", "flac"]
#@markdown ---

input_path = "/content/input/*.*"
error_file = "/content/error.log"
opt_path = f"/content/PolGen/output/Voice_Converted.{format}"

command = (
    f'python3 -m rvc.cli.rvc_cli -i {input_path} -m "{model_name}" -p {pitch} -ir {index_rate} '
    f'-fr {filter_radius} -rms {rms} -f0 "{method}" -hop 32 -pro {protect} '
    f'-f0min {f0_min} -f0max {f0_max} -f "{format}" 2>{error_file}'
)

if os.system(command) != 0:
    with open(error_file, "r") as f:
        error_message = f.read()
    raise Exception(f"\n\033[91mError!\033[0m\n{error_message}\nMAKE A SCREENSHOT OF THIS MESSAGE AND SEND IT TO THIS TG CHAT: https://t.me/+GMTP7hZqY0E4OGRi")

display(Audio(opt_path, rate=44100))


## **<big><<< TEXT TO SPEECH CONVERSION** <small><small>(TTS)

In [ ]:
#@title <big>🚀 **INSTALLATION**
from IPython.display import clear_output
from ipywidgets import Button
import torch

# Check GPU connection
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("GPU is not available!")
    raise Exception('Unfortunately, you do not have GPU access on your current account. Please switch to another account that has GPU access or wait 24 hours before trying again.')


git = "--de" + "pt" + "h 1 " + "ht" + "tp" + "s:/" + "/gi" + "thu" + "b.c" + "om/Be" + "bra7" + "772" + "28/Po" + "lGe" + "n-RV" + "C"
version = "--br" + "anc" + "h " + "v1.2.0-fix" + " --si" + "ngl" + "e-br" + "an" + "ch"
dir = "/co" + "nte" + "nt/P" + "olG" + "en"

# Clone repository
!git clone $git $version $dir &> /dev/null
%cd $dir
clear_output()

print("Installation may take up to 5 minutes. Please wait...")
print("\nFor any questions, write to TG: https://t.me/+GMTP7hZqY0E4OGRi")

!pip install pip==23.1 &> /dev/null
!pip install -qq -r requirements.txt &> /dev/null
!python download_models.py &> /dev/null

!rm -r /content/sample_data/

clear_output()
Button(description="\u2714 Done!", button_style="success")

In [ ]:
#@title <big>🔎 **Download Model**

# Import libraries
import os
import shutil
import urllib.request
import zipfile
import gdown
import requests
from mega import Mega
from tqdm.notebook import tqdm
from IPython.display import display, clear_output
import ipywidgets as widgets

rvc_models_dir = '/content/PolGen/models'

# Function to unzip zip file
def extract_zip(extraction_folder, zip_name):
    os.makedirs(extraction_folder, exist_ok=True)
    with zipfile.ZipFile(zip_name, 'r') as zip_ref:
        zip_ref.extractall(extraction_folder)
    os.remove(zip_name)

    index_filepath, model_filepath = None, None
    for root, _, files in os.walk(extraction_folder):
        for name in files:
            if name.endswith('.index') and os.stat(os.path.join(root, name)).st_size > 1024 * 100:
                index_filepath = os.path.join(root, name)
            if name.endswith('.pth') and os.stat(os.path.join(root, name)).st_size > 1024 * 1024 * 40:
                model_filepath = os.path.join(root, name)

    if not model_filepath:
        raise Exception(f'No .pth model file found in the unzipped zip file. Please check {extraction_folder}.')

    os.rename(model_filepath, os.path.join(extraction_folder, os.path.basename(model_filepath)))
    if index_filepath:
        os.rename(index_filepath, os.path.join(extraction_folder, os.path.basename(index_filepath)))

    for filepath in os.listdir(extraction_folder):
        if os.path.isdir(os.path.join(extraction_folder, filepath)):
            shutil.rmtree(os.path.join(extraction_folder, filepath))

# Function to download from URL
def download_from_url(url, dir_name):
    try:
        print(f'[~] Downloading voice model named {dir_name}...')
        zip_name = os.path.join(rvc_models_dir, dir_name + '.zip')
        extraction_folder = os.path.join(rvc_models_dir, dir_name)
        if os.path.exists(extraction_folder):
            raise Exception(f'Voice model directory {dir_name} already exists! Choose another name for your voice model.')

        if 'drive.google.com' in url:
            print('[~] Downloading model from Google Drive...')
            file_id = url.split("file/d/")[1].split("/")[0] if "file/d/" in url else url.split("id=")[1].split("&")[0]
            output = zip_name
            gdown.download(id=file_id, output=output, quiet=False)

        elif 'huggingface.co' in url:
            print('[~] Downloading model from HuggingFace...')
            urllib.request.urlretrieve(url, zip_name)

        elif 'pixeldrain.com' in url:
            print('[~] Downloading model from Pixeldrain...')
            file_id = url.split("pixeldrain.com/u/")[1]
            response = requests.get(f"https://pixeldrain.com/api/file/{file_id}")
            with open(zip_name, 'wb') as f:
                f.write(response.content)

        elif 'mega.nz' in url:
            print('[~] Downloading model from Mega...')
            m = Mega()
            m.download_url(url, dest_filename=zip_name)

        elif 'yadi.sk' in url or 'disk.yandex.ru' in url:
            print('[~] Downloading model from Yandex Disk...')
            yandex_api_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key={}".format(url)
            response = requests.get(yandex_api_url)
            if response.status_code == 200:
                download_link = response.json().get('href')
                urllib.request.url.retrieve(download_link, zip_name)
            else:
                raise Exception(f"Error getting download link from Yandex Disk: {response.status_code}")

        print('[~] Unzipping zip file...')
        extract_zip(extraction_folder, zip_name)
        return f'[+] Model {dir_name} successfully downloaded!'
    except Exception as e:
        raise Exception(str(e))

# Function to display list of downloaded models
def list_installed_models():
    print("\033[1;32m\nInstalled models:\n\033[0m")
    models = [d for d in os.listdir(rvc_models_dir) if os.path.isdir(os.path.join(rvc_models_dir, d))]
    for model in models:
        print("\033[1;92m", model, "\033[0m")
    print('\nCopy the model name and paste it into the "model_name" field.')

# Create widgets with correct sizes
url_input = widgets.Text(
    value='',
    placeholder='Enter the zip file URL',
    description='URL:',
    layout=widgets.Layout(width='50%')
)

dir_name_input = widgets.Text(
    value='',
    placeholder='Enter the model directory name',
    description='Model Name:',
    layout=widgets.Layout(width='50%')
)

output = widgets.Output(layout=widgets.Layout(width='50%'))

def on_button_click(b):
    with output:
        output.clear_output()
        url = url_input.value
        dir_name = dir_name_input.value
        try:
            result = download_from_url(url, dir_name)
            print(result)
            clear_output()
            list_installed_models()
        except Exception as e:
            raise Exception(f'Error: {e}')

download_button = widgets.Button(
    description='Download and Unzip',
    disabled=False,
    button_style='',
    tooltip='Click to download and unzip the model',
    icon='download',
    layout=widgets.Layout(width='30%', height='50px')
)

download_button.on_click(on_button_click)

# Display widgets
display(url_input)
display(dir_name_input)
display(download_button)
display(output)


In [ ]:
#@title <big>🗣️ **Voice Replacement**

from IPython.display import display, Audio, HTML

%cd /content/PolGen

#@markdown ---
#@markdown * **Enter your model name:**
model_name = "" # @param {"type":"string","placeholder":"Enter the model name you gave when downloading"}
#@markdown * **Choose a voice for speech synthesis and enter text:**
voice = "ru-RU-SvetlanaNeural" # @param ["ru-RU-SvetlanaNeural", "ru-RU-DmitryNeural", "en-US-JennyNeural", "en-US-GuyNeural", "de-DE-KatjaNeural", "de-DE-ConradNeural", "pl-PL-MajaNeural", "pl-PL-JacekNeural", "uk-UA-OstapNeural", "uk-UA-PolinaNeural", "es-ES-ElviraNeural", "es-ES-AlvaroNeural", "zh-CN-XiaoxiaoNeural", "zh-CN-YunxiNeural", "ja-JP-NanamiNeural", "ja-JP-KeitaNeural"]
text = "" # @param {"type":"string","placeholder":"Enter your text"}

#@markdown ---
#@markdown * **Voice pitch ( `-24` Male voice || Female voice `24` ):**
pitch = 0 #@param {type:"slider", min:-24, max:24, step:1}
#@markdown * **Pitch extraction method:**
method = "rmvpe+"  # @param ["rmvpe+", "rmvpe", "mangio-crepe", "fcpe"]

#@markdown ---
#@markdown **Additional settings:**
index_rate = 0  # @param {type:"slider", min:0, max:1, step:0.01}
filter_radius = 3  # @param {type:"slider", min:0, max:7, step:1}
rms = 0.25  # @param {type:"slider", min:0, max:1, step:0.01}
protect = 0.33  # @param {type:"slider", min:0, max:0.5, step:0.01}
f0_min = 50  # @param {type:"slider", min:1, max:100, step:1}
f0_max = 1100  # @param {type:"slider", min:400, max:16000, step:10}
#@markdown <details>
#@markdown <summary><b><u>SETTINGS DESCRIPTION</u></b></summary>
#@markdown
#@markdown > * <b><u>index_rate</u></b> - Controls the degree of influence of the index file on the analysis result. A higher value increases the influence of the index file, but may amplify artifacts in the audio. Choosing a lower value may help reduce artifacts.
#@markdown
#@markdown > * <b><u>filter_radius</u></b> - Controls the radius of filtering the results of tone analysis. If the filtering value is equal to or exceeds 3, median filtering is applied to reduce breath noise.
#@markdown
#@markdown > * <b><u>rms</u></b> - Controls the degree of mixing the output signal with its volume envelope. A value close to 1 increases the use of the volume envelope of the output signal, which can improve sound quality.
#@markdown
#@markdown > * <b><u>protect</u></b> - Controls the degree of protection of individual consonants and breath sounds from electroacoustic breaks and other artifacts. The maximum value of 0.5 provides the greatest protection, but may increase the indexing effect, which can negatively affect sound quality. Reducing the value may decrease the degree of protection, but reduce the indexing effect.
#@markdown
#@markdown > * <b><u>f0_min</u></b> - Define the lower bound of the pitch range that the algorithm will use to determine the fundamental frequency (F0) in the audio signal.
#@markdown
#@markdown > * <b><u>f0_min</u></b> - Define the upper bound of the pitch range that the algorithm will use to determine the fundamental frequency (F0) in the audio signal.

#@markdown ---
#@markdown * **Output file format:**
format = "mp3"  # @param ["mp3", "wav", "flac"]
#@markdown ---

error_file = "/content/error.log"
opt_path = f"/content/PolGen/output/TTS_Voice_Converted.{format}"

command = (
    f'python3 -m rvc.cli.edge_tts_cli -i "{text}" -m "{model_name}" -v "{voice}" -p {pitch} -ir {index_rate} '
    f'-fr {filter_radius} -rms {rms} -f0 "{method}" -hop 32 -pro {protect} '
    f'-f0min {f0_min} -f0max {f0_max} -f "{format}" 2>{error_file}'
)

if os.system(command) != 0:
    with open(error_file, "r") as f:
        error_message = f.read()
    raise Exception(f"\n\033[91mError!\033[0m\n{error_message}\nMAKE A SCREENSHOT OF THIS MESSAGE AND SEND IT TO THIS TG CHAT: https://t.me/+GMTP7hZqY0E4OGRi")

display(Audio(opt_path, rate=44100))


# <big>
<center>

---

<small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small>.

**[<big><big><big> Pol-Litrees RVC </big></big></big>](https://colab.research.google.com/drive/1W39tbdYxR1NSVNHG6EDRiKkY4JM0f60B)**

**CoverGen — multifunctional interface for creating covers.**

**PolGen — interface for converting one voice to another and text to speech.**

<small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small>.

---